# 📘 Proyecto AmbiWorld – Extracción de Frases con Palabras Chinas Ambiguas

In [ ]:
# Reinstalar numpy en una versión compatible con gensim y tensorflow
!pip install numpy==1.26.4
!pip install gensim==4.3.3
!pip install jieba
!pip install scikit-learn
!pip install pandas

In [ ]:
import numpy as np
import pandas as pd
import jieba
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:

#Descargar corpus de Tatoeba
!wget https://downloads.tatoeba.org/exports/sentences.csv -O sentences.csv

#Cargar solo frases en chino
import pandas as pd
df = pd.read_csv("sentences.csv", sep='\t', names=["id", "lang", "sentence"])
df_cmn = df[df["lang"] == "cmn"].reset_index(drop=True)


#diccionario de palabras ambiguas
palabras_ambiguas = list(diccionario.keys())
print(f"Usando {len(palabras_ambiguas)} palabras ambiguas")

#Buscar frases con palabras ambiguas
frases_filtradas = []
for _, row in df_cmn.iterrows():
    for palabra in palabras_ambiguas:
        if palabra in row["sentence"]:
            frases_filtradas.append({
                "frase": row["sentence"],
                "palabra": palabra,
                "significado_correcto": ""  #a rellenar
            })
            break  #evita duplicados

#Guardamos frases en un CSV para etiquetar
df_resultado = pd.DataFrame(frases_filtradas)
df_resultado.to_csv("frases_para_etiquetar.csv", index=False)
print(f"Se encontraron {len(df_resultado)} frases con palabras ambiguas.")


In [ ]:
diccionario = {palabra: idx for idx, palabra in enumerate(set(tokens))}
print(diccionario)


In [ ]:
#traduccion de frases chinas
#Instalación de dependencias
!pip install transformers sentencepiece

In [ ]:
#Importación de librerías
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm

In [ ]:
#Cargar el archivo de frases
file_path = "/content/frases_para_etiquetar.csv"  # Cambia esto si tu archivo está en otra ubicación
df = pd.read_csv(file_path)

In [ ]:
#chino -> inglés
from transformers import MarianMTModel, MarianTokenizer

# Cargar modelo zh-en
zh_en_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
zh_en_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

# Cargar modelo en-es
en_es_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-es")
en_es_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")

# Función en cadena
def traducir_zh_es(texto):
    tokens_zh = zh_en_tokenizer([texto], return_tensors="pt", padding=True, truncation=True)
    traducido_en = zh_en_model.generate(**tokens_zh)
    texto_en = zh_en_tokenizer.decode(traducido_en[0], skip_special_tokens=True)

    tokens_en = en_es_tokenizer([texto_en], return_tensors="pt", padding=True, truncation=True)
    traducido_es = en_es_model.generate(**tokens_en)
    return en_es_tokenizer.decode(traducido_es[0], skip_special_tokens=True)

In [ ]:
#ver el progreso de todas las frases llevara 10 horas( asi que tomaresmos 500)
from tqdm import tqdm
tqdm.pandas()

df["traduccion_es"] = df["frase"].progress_apply(traducir_zh_es)

In [ ]:
from tqdm import tqdm
tqdm.pandas()

# llavara mucho tiempo ver el progreso de todas, entonces solo tomamos 500 frases para probar
df_sample = df.sample(500).copy()

# Aplicamos la traducción solo a estas
df_sample["traduccion_es"] = df_sample["frase"].progress_apply(traducir_zh_es)

# Guardamos el resultado para revisarlo
df_sample.to_csv("frases_traducidas_500.csv", index=False)
print("Archivo guardado: frases_traducidas_500.csv")

In [ ]:
#con las 500 frases traducidas, podemos:
#Aplicar el sistema de etiquetado automático con spaCy y Significados_es.
#Ver qué tan bien se predicen los significados.
#Ey asi entrenar un primer modelo de clasificación.

In [ ]:
#etiquetamos automaticamente  los significados de palabras ambiguas
#usando la traducción al español y spaCy para similitud semántica.
!pip install spacy
!python -m spacy download es_core_news_md

In [ ]:
# Cargar el modelo de spaCy en español
import spacy
nlp = spacy.load("es_core_news_md")

# Cargar el archivo con traducción
df = pd.read_csv("frases_traducidas_500.csv")

# Mostrar columnas para verificar
print(df.columns)

In [ ]:
#funcion para predecir el significado correcto
# Esta función compara cada significado con la traducción al español
def predecir_significado(fila):
    if pd.isna(fila["traduccion_es"]):
        return "NO_DISPONIBLE"

    significados = [s.strip() for s in fila["traduccion_es"].split(";")]
    traduccion = fila["traduccion_es"]
    doc_trad = nlp(traduccion)

    max_score = 0
    mejor_significado = "NO_DETECTADO"

    for sig in significados:
        doc_sig = nlp(sig)
        score = doc_trad.similarity(doc_sig)
        if score > max_score:
            max_score = score
            mejor_significado = sig

    return mejor_significado

In [ ]:
#aplicamos la funcion al dataframe
tqdm.pandas()
df["significado_correcto"] = df.progress_apply(predecir_significado, axis=1)

In [ ]:
#guardamos el archivo etiquetado
df.to_csv("frases_etiquetadas.csv", index=False)
print(" Archivo guardado como frases_etiquetadas.csv")

In [ ]:
#pasamos a entrenar el modelo